**COVID 19 - Overview & Visualization**

---
updated daily


In [ ]:
#@title
# Data Processing
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium as folium
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff


tf.random.set_seed(17042020)

Inputdatenquelle: [ECDC](https://www.ecdc.europa.eu/en) is an EU agency aimed at strengthening Europe's defences against infectious diseases. The core functions cover a wide spectrum of activities: surveillance, epidemic intelligence, response, scientific advice, microbiology, preparedness, public health training, international relations, health communication, and the scientific journal Eurosurveillance.


---


[Daten-Download-Link](https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx)

In [ ]:
#@title

try:
  #covid cases
  data = pd.read_excel("https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx")

  weekly_data = pd.read_excel("https://opendata.ecdc.europa.eu/covid19/subnationalcaseweekly/xlsx")

  cumulated_data = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-table-data.csv")

  #additional covid data
  url1 = "https://raw.githubusercontent.com/LuijiDarkSia/covid/main/data/covid_19_clean_complete.csv"
  url2 = "https://raw.githubusercontent.com/LuijiDarkSia/covid/main/data/covid_19_clean_complete_for.csv"
  
  #geodata
  url3 = "https://raw.githubusercontent.com/LuijiDarkSia/covid/main/data/world_country_and_usa_states_latitude_and_longitude_values.csv"

  full_table = pd.read_csv(url1)
  coordinates = pd.read_csv(url2)
  geo_coordinates = pd.read_csv(url3)
  print("Data download was successful")
except:
  print("Data download failed")

Data download was successful


In [ ]:
#@title
geo_coordinates = geo_coordinates[["country_code", "latitude", "longitude"]]
geo_coordinates = geo_coordinates.rename(columns={'country_code':'geoId', "latitude":"Lat", "longitude":"Long"})
#data.loc[data['geoId'] == "UK"]
#geo_coordinates.loc[geo_coordinates['geoId'] == "GB"]
geo_coordinates.replace('GB', 'UK', inplace=True)
#geo_coordinates.head()

In [ ]:
#@title
coordinates = coordinates[["Country/Region", "Lat", "Long"]]

In [ ]:
#@title
coordinates = coordinates.rename(columns={'Country/Region':'countriesAndTerritories'})
#coordinates

In [ ]:
#@title
# Grouped by day, country
# =======================

# Cleaning data
# =============

full_grouped = data.groupby(['dateRep', 'geoId'])['cases', 'deaths'].sum().reset_index()

# new cases ======================================================
temp = full_grouped.groupby(['geoId', 'dateRep'])['cases', 'deaths']
temp = temp.sum().diff().reset_index()

mask = temp['geoId'] != temp['geoId'].shift(1)

temp.loc[mask, 'cases'] = np.nan
temp.loc[mask, 'deaths'] = np.nan

# renaming columns
temp.columns = ['geoId', 'dateRep', 'New cases', 'New deaths']
# =================================================================

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['geoId', 'dateRep'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['New cases', 'New deaths']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

full_grouped['cumsum_cases'] = full_grouped.groupby('geoId')['cases'].transform(pd.Series.cumsum)
full_grouped['cumsum_deaths'] = full_grouped.groupby('geoId')['deaths'].transform(pd.Series.cumsum)

#full_grouped

In [ ]:
#@title
# Group by Day
day_wise = full_grouped.groupby("dateRep")["cases", "deaths", "cumsum_cases", "cumsum_deaths"].sum().reset_index()

# Deathrate
day_wise["Deaths / 100 Cases"] = round((day_wise["deaths"]/day_wise["cases"])*100, 2)

#Fill NaN
cols = ["Deaths / 100 Cases"]
day_wise[cols] = day_wise[cols].fillna(0)


#day_wise.tail()

In [ ]:
#@title
# Group by Country
country_wise = data.groupby("geoId")["cases", "deaths"].sum().reset_index()

#Adding Population Data
# select only population
pop = data[["geoId", "popData2019"]]

country_wise.replace('United_States_of_America', 'United States', inplace=True)

#Removing Duplicates
pop = pop.drop_duplicates(subset ="geoId")
#coordinates = coordinates.drop_duplicates(subset ="geoId")


#Merging the Population Data to the Country Dataframe
country_wise = pd.merge(country_wise, pop, on="geoId", how='left')

#Merging the Coordinates to the Country Dataframe
country_wise = pd.merge(country_wise, geo_coordinates, on="geoId", how='left')

#Merging Country Names to Country Dataframe
data_temp = data[["geoId", "countriesAndTerritories"]]
data_temp = data_temp.drop_duplicates(subset ="geoId")
country_wise = pd.merge(country_wise, data_temp, on="geoId", how='left')

# per 100 cases
country_wise["Deaths / 100 Cases"] = round((country_wise["deaths"]/country_wise["cases"])*100, 2)

#Fill NaN
cols = ["Deaths / 100 Cases"]
country_wise[cols] = country_wise[cols].fillna(0)

country_wise.replace('United_States_of_America', 'United States', inplace=True)

#country_wise

In [ ]:
#@title
# Dealing with missing values

country_wise_for_map = country_wise.dropna()

In [ ]:
#@title
# Map Cases per Country

m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(country_wise_for_map)):
    folium.Circle(
        location=[country_wise_for_map.iloc[i]['Lat'], country_wise_for_map.iloc[i]['Long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(country_wise_for_map.iloc[i]['geoId'])+
                    '<li><bold>Cases : '+str(country_wise_for_map.iloc[i]['cases'])+
                    '<li><bold>Deaths : '+str(country_wise_for_map.iloc[i]['deaths']),
              radius=int(country_wise_for_map.iloc[i]['cases'])**0.9).add_to(m)
m

In [ ]:
#@title
# Grouped by day, country
# =======================

# Cleaning data
# =============

full_grouped = data.groupby(['dateRep', 'countriesAndTerritories'])['cases', 'deaths'].sum().reset_index()

# new cases ======================================================
temp = full_grouped.groupby(['countriesAndTerritories', 'dateRep'])['cases', 'deaths']
temp = temp.sum().diff().reset_index()

mask = temp['countriesAndTerritories'] != temp['countriesAndTerritories'].shift(1)

temp.loc[mask, 'cases'] = np.nan
temp.loc[mask, 'deaths'] = np.nan

# renaming columns
temp.columns = ['countriesAndTerritories', 'dateRep', 'New cases', 'New deaths']
# =================================================================

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['countriesAndTerritories', 'dateRep'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['New cases', 'New deaths']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

full_grouped['cumsum_cases'] = full_grouped.groupby('countriesAndTerritories')['cases'].transform(pd.Series.cumsum)
full_grouped['cumsum_deaths'] = full_grouped.groupby('countriesAndTerritories')['deaths'].transform(pd.Series.cumsum)

full_grouped.replace('United_States_of_America', 'United_States', inplace=True)


In [ ]:
#@title
# Over the time
import datetime as dt
fig = px.choropleth(full_grouped, locations="countriesAndTerritories", locationmode='country names', color=np.log(full_grouped["cumsum_cases"]), 
                    hover_name="countriesAndTerritories", animation_frame=full_grouped["dateRep"].dt.strftime('%Y-%m-%d'),
                    title='Reported Cases over time', color_continuous_scale=px.colors.sequential.Magenta)
fig.update(layout_coloraxis_showscale=False)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title
# Cases
fig_c = px.choropleth(country_wise, locations="countriesAndTerritories", locationmode='country names', 
                      color=np.log(country_wise["cases"]), hover_name="countriesAndTerritories", hover_data=['cases'])

# Deaths
temp = country_wise[country_wise['deaths']>0]
fig_d = px.choropleth(temp, locations="countriesAndTerritories", locationmode='country names',
                      color=np.log(temp["deaths"]), hover_name="countriesAndTerritories", hover_data=['deaths'])

# Plot
fig = make_subplots(rows=1, cols=2, subplot_titles = ['cases', 'deaths'],
                    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]])

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update(layout_coloraxis_showscale=False)

fig.show()

In [ ]:
#@title
fig_c = px.bar(day_wise, x="dateRep", y="cases")
fig_d = px.bar(day_wise, x="dateRep", y="deaths")

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('Reported Confirmed cases per day', 'Deaths reported per day'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [ ]:
#@title
fig_c = px.bar(day_wise, x="dateRep", y="cumsum_cases")
fig_d = px.bar(day_wise, x="dateRep", y="cumsum_deaths")

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('Reported Confirmed cases per day', 'Deaths reported per day'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [ ]:
#@title
fig_1 = px.line(day_wise, x="dateRep", y="Deaths / 100 Cases")

fig = make_subplots(rows=1, cols=3, shared_xaxes=False, 
                    subplot_titles=('Deaths / 100 Cases'))

fig.add_trace(fig_1['data'][0], row=1, col=1)

fig.update_layout(height=480)
fig.show()

In [30]:
#@title
fig = px.scatter(country_wise.sort_values('deaths', ascending=False).iloc[:30, :], 
                 x='cases', y='deaths', color='geoId', size='cases', height=700,
                 text='geoId', log_x=True, log_y=True, title='Deaths vs Confirmed (Scale is in log10)')
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
#@title
fig = px.bar(full_grouped, x="dateRep", y="cases", color='countriesAndTerritories', height=600,
             title='Cases reported per day', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

# =========================================

fig = px.bar(full_grouped, x="dateRep", y="deaths", color='countriesAndTerritories', height=600,
             title='Deaths report per day', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title
fig = px.bar(full_grouped, x="dateRep", y="cumsum_cases", color='countriesAndTerritories', height=600,
             title='Cases reported per day', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

# =========================================

fig = px.bar(full_grouped, x="dateRep", y="cumsum_deaths", color='countriesAndTerritories', height=600,
             title='Deaths report per day', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title
fig = px.pie(country_wise, values = 'cases',names='geoId', height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    

fig.show()

In [ ]:
#@title Standardtext für Titel
cumulated_data.head()

In [ ]:
#@title Standardtext für Titel
import plotly.express as px

cumulated_data_temp = cumulated_data[1:]

fig = px.choropleth(cumulated_data_temp, locations="Name", locationmode='country names', color='Cases - cumulative total per 1 million population', color_continuous_scale="inferno", range_color=(0, 150000), hover_name="Name", projection="orthographic")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#@title Standardtext für Titel
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/LuijiDarkSia/covid/main/data/landkreise_simplify0.geojson') as response:
    counties = json.load(response)

df = pd.read_csv("https://opendata.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0.csv")
df['RS'] = df['RS'].apply(str)
df['RS'] = df['RS'].str.zfill(5)

df_temp = df[400:]
df = df[:400]
list_temp = df_temp.mean(axis = 0, skipna = True)
list_temp["cases7_per_100k"] = df_temp["cases7_per_100k"].mean(axis = 0, skipna = True) 
list_temp["RS"] = 11000
list_temp["GEN"] = "Berlin"
df.loc[len(df)] = list_temp


counties["features"][400]["properties"]

In [ ]:
#@title Standardtext für Titel
df

In [ ]:
#!pip install plotly==4.14.3

In [31]:
#@title Standardtext für Titel
import plotly.express as px
fig = px.choropleth(data_frame=df,
                    geojson=counties,
                    color='cases7_per_100k',
                    locations='RS',
                    featureidkey="properties.RS",
                    color_continuous_scale='viridis',
                    center={"lat": 51, "lon": 9},
                    hover_data=["GEN", "cases_per_100k", "death7_lk"],
                    projection="mercator")



fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.update_traces(hovertemplate=f'Landkreise: %{df["GEN"]} <br>Cases 7 Days: %{df["cases_per_100k"]}') #
fig.show()